In [ ]:
#data wrangling
import numpy as np
import pandas as pd
import datetime

#plotting
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

#ML libraries
#TODO: Sort out which ones we really need
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

#other utilities
import os

In [ ]:
data_dir = os.fsencode('./data/')

In [ ]:
#read in he CSV files containing the data
#TODO: Abstract and build as function for util_pkg
df_dict = {}
for filename in os.listdir(data_dir):
    filepath_str = str(os.fsdecode(os.path.join(data_dir,filename)))
    df_name_str = str(os.fsdecode(os.path.splitext(filename)[0]))
    df_dict[df_name_str] = pd.read_csv(filepath_str, sep = '\t')
    print(df_name_str)
    

In [ ]:
#get an overview of the data we just pulled and drop unnecessary ones
for df in df_dict:
    print(df)
    print(df_dict[df].shape)
    display(df_dict[df].head())

In [ ]:
def check_null_cols(df_dict):
    #TODO: write a docstring
    
    for df in df_dict:
        print(df)
        display(pd.DataFrame((np.sum(df_dict[df].isnull())/df_dict[df].shape[0])))

check_null_cols(df_dict)

In [ ]:
# drop tags from course-meta... too many nulls
# TODO: also drop entries from course and course meta with catch < max(catch)
# for the others just drop rows missing value

course_meta_df = df_dict['course-meta'].\
                    drop(columns = ['tag', 'catch']).\
                    dropna(subset = ['firstClear'], axis = 0).\
                    drop_duplicates()

course_df = df_dict['courses'].\
                drop(columns = ['thumbnail', 'image', 'title']).\
                dropna(subset = ['maker'], axis = 0).\
                drop_duplicates()

course_complete_df = course_meta_df.merge(course_df, on = 'id')

players_df = df_dict['players'].drop(columns = ['image', 'name']).\
                rename(columns= {'id' : 'player_id'}).\
                drop_duplicates()

players_df

In [ ]:
df_dict['course-meta'].sort_values('id')

In [ ]:
#get player flags
course_complete_df = course_complete_df.merge(players_df,
                                             how = 'left',
                                             left_on = 'maker',
                                             right_on = 'player_id')

print(course_complete_df.shape)
print(course_complete_df.columns)

In [ ]:
course_complete_df.head()

In [ ]:
country_counts = course_complete_df.flag.value_counts()
difficulty_counts = course_complete_df.difficulty.value_counts() 

In [ ]:
(country_counts/course_complete_df.shape[0]).sort_values().plot(kind = 'bar')
plt.title("Share of Levels Created by Country");

In [ ]:
(difficulty_counts/course_complete_df.shape[0]).sort_values().plot(kind = 'bar')
plt.title("Share of Levels Created by Difficulty");

In [ ]:
print(course_complete_df.creation.max(), course_complete_df.creation.min())

In [ ]:
course_complete_df.flag.unique()

In [ ]:
players_df.flag.unique()

In [ ]:
#extract number of courses created and add as extra column


course_complete_df.courses_by_maker = course_complete_df.groupby('maker').transform('count')

course_complete_df['creation'] = pd.to_datetime(course_complete_df.creation)

course_complete_df['normalized_age'] = (course_complete_df.creation - course_complete_df.creation.min()).dt.days

In [ ]:
# print(course_complete_df.shape)
course_complete_df.drop_duplicates().head()

In [ ]:
course_clean_df = course_complete_df.unique().drop(columns = 
                ['firstClear', 'id', 'maker', 'player_id', 'clears', 'creation'])

# also drop clears... the info is not needed as we have 
# clears = attempts*(clearRate/100)

# we're dropping maker here although it might be useful...
# use somewhere else in analysis
# we should also extract creation in a meaningful way.

course_clean_df.head()